<a href="https://colab.research.google.com/github/raizo2000/0003/blob/master/perceptron_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import random
import numpy as np

Clase Perceptrón

Aquí creamos una clase para el Perceptrón que podemos utilizar para instaurar cualquier número de instancias de Perceptrón.

In [25]:
class Perceptron:
    
    def __init__(self, num_inputs, act='sigmoid'):
        self.weights = []
        self.num_inputs = num_inputs
        self.act = act # definir la función de activación con sigmoide por defecto
        for _ in range(0, num_inputs):
            self.weights.append(random.random() * 2 - 1)
        print(self.weights)
            
    def get_weights(self):
        return self.weights
        
    def feed_forward(self, inputs):
        self.inputs = inputs
        sum = 0
        
        # multiplicar las entradas por los pesos y sumarlos
        for i in range(0, self.num_inputs):
            sum += self.weights[i] * inputs[i]
            
        # 'activar' la suma y obtener la derivada
        self.output, self.output_prime = self.activate(sum)
        return self.output
    
    def activate(self, x):
        if (self.act == 'sigmoid'):
            activation = self.sigmoid(x)
            activation_prime = activation * (1 - activation)
        else:
            activation = self.step(x)
            activation_prime = 1 # utilice 1 ya que la activación de los pasos no es diferenciable
        return activation, activation_prime
    
    def sigmoid(self, x):
        return 1/(1 + np.exp(-x))
    
    def step(self, x):
        if x > 0:
            return 1
        return 0
    
    def backward_pass(self, error):
        learning_rate = 0.01 # hiperparámetro
        back_error = [] # cada elemento de la lista representa la cantidad de error a enviar hacia atrás a lo largo de esa conexión
        for i in range(0, self.num_inputs):
            back_error.append(error * self.output_prime * self.weights[i])
            self.weights[i] -= error * self.output_prime * self.inputs[i] * learning_rate
        return back_error

Perceptrón simple para clasificar un punto por encima o por debajo de una línea

Comienza con una línea que pasa por el origen.

In [26]:

def line(x):
    return 0.5 * x

In [27]:
p = Perceptron(2, act='step')
weights = p.get_weights()
print(sum(weights))

[-0.9625337599048984, 0.49159858114183397]
-0.47093517876306445


In [28]:

x_coord = random.random() * 1000
y_coord = random.random() * 1000
line_y = line(x_coord)

print(x_coord, y_coord)
print(x_coord, line_y)

if y_coord > line_y:
    answer = 1
else:
    answer = 0
    
print(answer)

865.3232035156077 475.7632889498449
865.3232035156077 432.66160175780385
1


In [29]:
guess = p.feed_forward([x_coord, y_coord])
p.backward_pass(guess - answer)
p.get_weights()

[7.690698275251179, 5.249231470640283]

In [30]:
for _ in range(0, 1000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    line_y = line(x_coord)
    
    if y_coord > line_y:
        answer = 1
    else:
        answer = 0
    
    guess = p.feed_forward([x_coord, y_coord])
    p.backward_pass(guess - answer)
    
p.get_weights()

[-19.165902273689323, 41.50300637090956]

In [31]:
correct = 0

for _ in range(0,1000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    line_y = line(x_coord)
    
    is_above = y_coord > line_y
    guess_above = p.feed_forward([x_coord, y_coord, 1])
    
    if (is_above == True and guess_above >= 0.5):
        correct += 1
    if (is_above == False and guess_above < 0.5):
        correct += 1

print(correct)

986


Añade un bias al nodo para manejar las líneas que no pasan por el origen.

In [32]:
def line(x):
    return 0.5 * x + 500

In [33]:
p = Perceptron(3, act='step')
weights = p.get_weights()

[0.07171145359447206, -0.7857625017219512, 0.31298329698722416]


In [34]:
for _ in range(0, 100000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    line_y = line(x_coord)
    
    if y_coord > line_y:
        answer = 1
    else:
        answer = 0
    
    guess = p.feed_forward([x_coord, y_coord, 1])
    p.backward_pass(guess - answer)
    
p.get_weights()

[-14.109206217248497, 8.830695422930614, -65.66701670300947]

In [35]:
correct = 0

for _ in range(0,1000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    line_y = line(x_coord)
    
    above = y_coord > line_y
    guess_above = p.feed_forward([x_coord, y_coord, 1])
    
    if (above == True and guess_above >= 0.5):
        correct += 1
    if (above == False and guess_above < 0.5):
        correct += 1

print(correct)

848


Network with 3 Perceptrons

In [36]:
a = Perceptron(3, act='step')
b = Perceptron(3, act='step')
c = Perceptron(3, act='step')

[-0.11527198509966996, 0.5510827367172897, 0.8142687038553427]
[-0.8181114163476555, 0.33173025869089545, -0.9557313608084479]
[-0.4558752343638608, -0.21976257892208007, -0.2516299029690561]


In [37]:
def network(first, second):
    a_out = a.feed_forward([first, second, 1])
    b_out = b.feed_forward([first, second, 1])
    c_out = c.feed_forward([a_out, b_out, 1])
    return c_out

In [38]:
for _ in range(0,1000000):
    first = random.choice([0, 1])
    second = random.choice([0, 1])
    c_out = network(first, second)
#     a_out = a.feed_forward([first, second, 1])
#     b_out = b.feed_forward([first, second, 1])
#     c_out = c.feed_forward([a_out, b_out, 1])
    
    if (first != second):
        answer = 1
    else:
        answer = 0
        
    back_error = c.backward_pass(c_out - answer)
    a.backward_pass(back_error[0])
    b.backward_pass(back_error[1])
    
print(a.get_weights())
print(b.get_weights())
print(c.get_weights())

[-0.1935368734456375, -0.03652452423155856, 0.03624114292078636]
[0.45870916718962323, 0.5558880891914151, -1.0128696313281889]
[-0.01587523436386052, -0.21976257892208007, 0.008370097030944006]


In [39]:

print(network(1, 1))
print(network(1, 0))
print(network(0, 1))
print(network(0, 0))

0
1
1
0


In [40]:

print('%f' % a.feed_forward([1, 1, 1]))
print('%f' % a.feed_forward([1, 0, 1]))
print('%f' % a.feed_forward([0, 1, 1]))
print('%f' % a.feed_forward([0, 0, 1]))
print("\n")
print('%f' % b.feed_forward([1, 1, 1]))
print('%f' % b.feed_forward([1, 0, 1]))
print('%f' % b.feed_forward([0, 1, 1]))
print('%f' % b.feed_forward([0, 0, 1]))
print("\n")
print('%f' % c.feed_forward([1, 1, 1]))
print('%f' % c.feed_forward([1, 0, 1]))
print('%f' % c.feed_forward([0, 1, 1]))
print('%f' % c.feed_forward([0, 0, 1]))

0.000000
0.000000
0.000000
1.000000


1.000000
0.000000
0.000000
0.000000


0.000000
0.000000
0.000000
1.000000
